<a href="https://colab.research.google.com/github/neculaluana/Twitter-emotion-analysis/blob/main/emotion_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving training.csv to training.csv
Saving validation.csv to validation.csv


In [3]:
import io
import pandas as pd
import numpy as np
df_train = pd.read_csv(io.BytesIO(uploaded['training.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
df_valid = pd.read_csv(io.BytesIO(uploaded['validation.csv']))

In [4]:
import re
def clean_tweet(tweet):
  tweet = re.sub(r'https?://[^ ]+', '', tweet)      #removes links
  tweet = re.sub(r'@[^ ]+', '', tweet)              #removes mentions
  tweet = re.sub(r'#', '', tweet)                   #removes hashtag symbol
  tweet = re.sub(r'([A-Za-z])\1{2,}', r'\1', tweet) #removes repeated characters ex: heeeeeeey
  tweet = re.sub(r'[^A-Za-z ]', '', tweet)          #removes unwanted characters and punctuation
  tweet = re.sub(r' 0 ', 'zero', tweet)             #transforms 0 to zero (it can influence emotion)
  tweet = tweet.lower()
  return tweet                             #lower-casing

df_train["text"].apply(lambda text: clean_tweet(text))
df_test["text"].apply(lambda text: clean_tweet(text))
df_valid["text"].apply(lambda text: clean_tweet(text))


0       im feeling quite sad and sorry for myself but ...
1       i feel like i am still looking at a blank canv...
2                          i feel like a faithful servant
3                       i am just feeling cranky and blue
4       i can have for a treat or if i am feeling festive
                              ...                        
1995    im having ssa examination tomorrow in the morn...
1996    i constantly worry about their fight against n...
1997    i feel its important to share this info for th...
1998    i truly feel that if you are passionate enough...
1999    i feel like i just wanna buy any cute make up ...
Name: text, Length: 2000, dtype: object

In [ ]:
df_train['label'] = df_train['label'].astype('category')
df_test['label'] = df_test['label'].astype('category')
df_valid['label'] = df_valid['label'].astype('category')

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.2 MB/s eta 0:00:00
